In [25]:
from typing import Tuple
import random

class Agent:
    def __init__(self, position: Tuple[int, int], opinion: int = 0):
        """
        Initialize an agent with a position and opinion.
        
        Args:
            position (Tuple[int, int]): Initial position (x, y)
            opinion (int): Initial opinion (0, 1, or 2)
        """
        self.position = position
        self.opinion = opinion
    
    def propose_move(self) -> Tuple[int, int]:
        """
        Propose a move to a random adjacent cell.
        
        Returns:
            Tuple[int, int]: Proposed new position (x, y)
        """
        x, y = self.position
        # Get all possible adjacent positions
        possible_moves = []
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            nx, ny = x + dx, y + dy
            possible_moves.append((nx, ny))
        
        # Return a random adjacent position
        return random.choice(possible_moves)
    
    def interact(self, other_agent: 'Agent') -> None:
        """
        Interact with another agent, potentially changing opinions.
        
        Args:
            other_agent (Agent): The agent to interact with
        """
        # If opinions are different, there's a chance they'll change
        if self.opinion != other_agent.opinion:
            # 50% chance for each agent to change their opinion
            if random.random() < 0.5:
                self.opinion = other_agent.opinion
            if random.random() < 0.5:
                other_agent.opinion = self.opinion 

In [26]:
import numpy as np
from typing import List, Tuple
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from agent import Agent

class Environment:
    def __init__(self, width: int = 100, height: int = 100):
        """
        Initialize the grid environment.
        
        Args:
            width (int): Width of the grid
            height (int): Height of the grid
        """
        self.width = width
        self.height = height
        self.grid = np.zeros((height, width), dtype=int)  # Initialize with 0's to represent empty cell
        self.agents: List[Agent] = []
        
    def add_agent(self, agent: Agent) -> bool:
        """
        Add an agent to the environment at its current position.
        
        Args:
            agent (Agent): The agent to add
            
        Returns:
            bool: True if agent was successfully added, False otherwise
        """
        x, y = agent.position
        if 0 <= x < self.width and 0 <= y < self.height and self.grid[y, x] == 0:
            self.grid[y, x] = 1  # Mark cell as occupied
            self.agents.append(agent)
            return True
        return False
    
    def remove_agent(self, agent: Agent) -> None:
        """
        Remove an agent from the environment.
        
        Args:
            agent (Agent): The agent to remove
        """
        if agent in self.agents:
            x, y = agent.position
            self.grid[y, x] = 0  # Mark cell as empty
            self.agents.remove(agent)
    
    def validate_move(self, agent, new_position: Tuple[int, int]) -> bool:
        """
        Validate if a move is possible.
        
        Args:
            agent: The agent trying to move
            new_position (Tuple[int, int]): Proposed new position (x, y)
            
        Returns:
            bool: True if move is valid, False otherwise
        """
        x, y = new_position
        return (0 <= x < self.width and 
                0 <= y < self.height and 
                self.grid[y, x] == 0)
    
    def move_agent(self, agent: Agent, new_position: Tuple[int, int]) -> bool:
        """
        Move an agent to a new position if the move is valid.
        
        Args:
            agent (Agent): The agent to move
            new_position (Tuple[int, int]): New position (x, y)
            
        Returns:
            bool: True if move was successful, False otherwise
        """
        if self.validate_move(agent, new_position):
            old_x, old_y = agent.position
            x, y = new_position
            self.grid[old_y, old_x] = 0  # Clear old position
            self.grid[y, x] = 1  # Mark new position as occupied
            agent.position = new_position
            return True
        return False
    
    def get_empty_cells(self) -> List[Tuple[int, int]]:
        """
        Get a list of all empty cell positions.
        
        Returns:
            List[Tuple[int, int]]: List of (x, y) coordinates of empty cells
        """
        return [(x, y) for y in range(self.height) for x in range(self.width) 
                if self.grid[y, x] == 0]

    def step(self) -> None:
        """
        Run one step of the simulation where each agent:
        1. Moves to a random adjacent cell if possible
        2. Interacts with any agents in adjacent cells
        """
        # Shuffle agents to randomize order of movement and interaction
        random.shuffle(self.agents)
        
        # Move agents
        for agent in self.agents:
            proposed_move = agent.propose_move()
            self.move_agent(agent, proposed_move)
        
        # Shuffle again before interactions to ensure random order
        random.shuffle(self.agents)
        
        # Handle interactions
        for agent in self.agents:
            x, y = agent.position
            # Check adjacent cells for other agents
            for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.width and 0 <= ny < self.height:
                    # Find any agent at this position
                    for other_agent in self.agents:
                        if other_agent.position == (nx, ny):
                            agent.interact(other_agent)

    def get_opinion_distribution(self) -> dict:
        """
        Get the current distribution of opinions among agents.
        
        Returns:
            dict: Dictionary mapping opinions to counts
        """
        opinion_counts = {}
        for agent in self.agents:
            opinion_counts[agent.opinion] = opinion_counts.get(agent.opinion, 0) + 1
        return opinion_counts 

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from IPython.display import HTML
from environment import Environment
from agent import Agent

def create_environment(width: int = 100, height: int = 100, num_agents: int = 1000) -> Environment:
    """
    Create and initialize the environment with agents.
    
    Args:
        width (int): Width of the grid
        height (int): Height of the grid
        num_agents (int): Number of agents to create
        
    Returns:
        Environment: The initialized environment
    """
    env = Environment(width, height)
    
    # Create agents with equal distribution of opinions
    opinions_per_agent = num_agents // 3
    for i in range(3):
        for _ in range(opinions_per_agent):
            # Get a random empty cell
            empty_cells = env.get_empty_cells()
            if empty_cells:
                position = empty_cells[np.random.randint(len(empty_cells))]
                agent = Agent(position, opinion=i)
                env.add_agent(agent)
    
    return env

def visualize_environment(env: Environment, show: bool = True) -> None:
    """
    Visualize the current state of the environment using matplotlib.
    Empty cells are white, agents are colored by their opinion.
    
    Args:
        env (Environment): The environment to visualize
        show (bool): Whether to display the plot
    """
    # Create a grid where:
    # 0 = empty (white)
    # 1-3 = different opinions (different colors)
    vis_grid = np.zeros((env.height, env.width))
    
    # Map agent positions and opinions to the visualization grid
    for agent in env.agents:
        x, y = agent.position
        vis_grid[y, x] = agent.opinion + 1  # +1 because 0 is reserved for empty cells
    
    # Create custom colormap
    colors = ['white', 'red', 'blue', 'green']  # white for empty, then colors for opinions
    max_opinion = int(vis_grid.max())  # Convert to int for indexing
    cmap = ListedColormap(colors[:max_opinion + 1])
    
    # Create the plot
    plt.figure(figsize=(10, 10))
    plt.imshow(vis_grid, cmap=cmap, vmin=0, vmax=len(colors)-1)
    plt.grid(True, which='both', color='black', linewidth=0.5)
    plt.xticks(range(env.width))
    plt.yticks(range(env.height))
    
    # Add a legend
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Empty')]
    for i, color in enumerate(colors[1:], 1):
        legend_elements.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, 
                                           label=f'Opinion {i-1}'))
    plt.legend(handles=legend_elements, loc='upper right')
    
    plt.title('Environment State')
    if show:
        plt.show()
    plt.close()

def animate_simulation(env: Environment, steps: int, interval: int = 500) -> None:
    """
    Create an animation of the simulation.
    
    Args:
        env (Environment): The environment to simulate
        steps (int): Number of steps to simulate
        interval (int): Time between frames in milliseconds
    """
    # Create figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), 
                                  gridspec_kw={'width_ratios': [2, 1]})
    
    # Create custom colormap
    colors = ['white', 'red', 'blue', 'green']
    cmap = ListedColormap(colors)
    
    # Initialize the grid plot
    vis_grid = np.zeros((env.height, env.width))
    for agent in env.agents:
        x, y = agent.position
        vis_grid[y, x] = agent.opinion + 1
    
    img = ax1.imshow(vis_grid, cmap=cmap, vmin=0, vmax=len(colors)-1)
    ax1.grid(True, which='both', color='black', linewidth=0.5)
    # Remove tick marks and labels
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('Agent Positions')
    
    # Add legend to grid plot
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Empty')]
    for i, color in enumerate(colors[1:], 1):
        legend_elements.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, 
                                           label=f'Opinion {i-1}'))
    ax1.legend(handles=legend_elements, loc='upper right')
    
    # Initialize the time series plot
    opinion_dist = env.get_opinion_distribution()
    time_data = [0]  # Time points
    opinion_data = {0: [opinion_dist.get(0, 0)],  # Opinion 0 counts
                   1: [opinion_dist.get(1, 0)],  # Opinion 1 counts
                   2: [opinion_dist.get(2, 0)]}  # Opinion 2 counts
    
    # Create lines for each opinion
    lines = []
    for i in range(3):
        line, = ax2.plot(time_data, opinion_data[i], color=colors[i+1], 
                       label=f'Opinion {i}', linewidth=2)
        lines.append(line)
    
    ax2.set_xlabel('Step')
    ax2.set_ylabel('Number of Agents')
    ax2.set_title('Opinion Distribution Over Time')
    ax2.set_ylim(0, len(env.agents))
    ax2.set_xlim(0, steps)
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.legend()
    
    # Add opinion distribution text
    opinion_text = ax1.text(0.02, 0.98, '', transform=ax1.transAxes, 
                          verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    
    def update(frame):
        env.step()
        # Update grid
        vis_grid = np.zeros((env.height, env.width))
        for agent in env.agents:
            x, y = agent.position
            vis_grid[y, x] = agent.opinion + 1
        img.set_array(vis_grid)
        
        # Update opinion distribution
        opinion_dist = env.get_opinion_distribution()
        time_data.append(frame + 1)
        for i in range(3):
            opinion_data[i].append(opinion_dist.get(i, 0))
            lines[i].set_data(time_data, opinion_data[i])
        
        # Update opinion text
        opinion_text.set_text('\n'.join([f'Opinion {k}: {v} agents' 
                                       for k, v in sorted(opinion_dist.items())]))
        
        # Update the figure title
        plt.suptitle(f'Step {frame + 1}/{steps}', y=0.95, fontsize=12, fontweight='bold')
        
        # Stop animation if we've reached the last frame
        if frame >= steps - 1:
            anim.event_source.stop()
        
        return [img, opinion_text] + lines
    
    anim = animation.FuncAnimation(fig, update, frames=steps, interval=interval, blit=True)
    plt.close(fig)
    return HTML(anim.to_jshtml()) # edit for notebook animation

def main():
    # Create and initialize the environment
    env = create_environment(width=50, height=50, num_agents=1000)
    
    # Run the simulation with animation
    anim_html = animate_simulation(env, steps=400, interval=50) # Specific to notebook version
    display(anim_html)

if __name__ == "__main__":
    main() 

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

New Configuration

In [21]:
from typing import Tuple, List
import random

class Agent:
    def __init__(self, position: Tuple[int, int], opinion: int = 0, is_high_integrity: bool = False):
        """
        Initialize an agent with a position, opinion, and integrity.
        
        Args:
            position (Tuple[int, int]): Initial position (x, y)
            opinion (int): Initial opinion (0, 1, or 2)
            is_high_integrity (bool): Whether this agent has fixed high integrity
        """
        self.position = position
        self.opinion = opinion
        self.is_high_integrity = is_high_integrity
        self.friends: List['Agent'] = []  # List of friend agents
        
        # Set integrity based on agent type
        if is_high_integrity:
            self.integrity = 1.0  # 100% integrity, never changes
        else:
            self.integrity = random.uniform(0.3, 0.7)  # Random initial integrity
    
    def add_friend(self, friend: 'Agent') -> None:
        """
        Add a friend to this agent's social circle.
        
        Args:
            friend (Agent): The agent to add as a friend
        """
        if friend not in self.friends and friend != self:
            self.friends.append(friend)
            # Make it bidirectional
            if self not in friend.friends:
                friend.friends.append(self)
    
    def remove_friend(self, friend: 'Agent') -> None:
        """
        Remove a friend from this agent's social circle.
        
        Args:
            friend (Agent): The agent to remove as a friend
        """
        if friend in self.friends:
            self.friends.remove(friend)
            # Make it bidirectional
            if self in friend.friends:
                friend.friends.remove(self)
    
    def add_friends(self, all_agents: List['Agent'], num_friends: int = 8) -> None:
        """
        Add friends to this agent with the same opinion.
        
        Args:
            all_agents (List[Agent]): All agents in the simulation
            num_friends (int): Maximum number of friends to add
        """
        # Filter out self and existing friends, and only include agents with same opinion
        available_agents = [agent for agent in all_agents 
                          if agent != self and agent not in self.friends and agent.opinion == self.opinion]
        
        # Add friends (all will have same opinion)
        friends_added = 0
        while friends_added < num_friends and available_agents:
            # Pick a random agent with same opinion
            friend = random.choice(available_agents)
            available_agents.remove(friend)
            
            # Add friend (bidirectional)
            self.add_friend(friend)
            friends_added += 1

    def propose_move(self) -> Tuple[int, int]:
        """
        Propose a move to a random adjacent cell.
        
        Returns:
            Tuple[int, int]: Proposed new position (x, y)
        """
        x, y = self.position
        # Get all possible adjacent positions
        possible_moves = []
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            nx, ny = x + dx, y + dy
            possible_moves.append((nx, ny))
        
        # Return a random adjacent position
        return random.choice(possible_moves)
    
    def interact(self, other_agent: 'Agent', is_friend_interaction: bool = False) -> None:
        """
        Interact with another agent, potentially changing opinions and integrity.
        
        Args:
            other_agent (Agent): The agent to interact with
            is_friend_interaction (bool): Whether this is a friend interaction (reduced effect)
        """
        # Determine if this is a positive or negative interaction (70% positive, 30% negative)
        is_positive = random.random() < 0.7
        
        # Determine if agents have the same opinion
        same_opinion = (self.opinion == other_agent.opinion)
        
        # Update integrity based on interaction type
        # High integrity agents can influence others but don't change their own integrity
        if not self.is_high_integrity:
            self._update_integrity(same_opinion, is_positive, is_friend_interaction)
        
        if not other_agent.is_high_integrity:
            other_agent._update_integrity(same_opinion, is_positive, is_friend_interaction)
        
        # Check if opinions should change based on integrity
        # High integrity agents can influence others but don't change their own opinions
        if not self.is_high_integrity:
            self._maybe_change_opinion(other_agent, is_friend_interaction)
        
        if not other_agent.is_high_integrity:
            other_agent._maybe_change_opinion(self, is_friend_interaction)
    
    def _update_integrity(self, same_opinion: bool, is_positive: bool, is_friend_interaction: bool = False) -> None:
        """
        Update agent's integrity based on interaction type.
        
        Args:
            same_opinion (bool): Whether the interaction was with same opinion
            is_positive (bool): Whether the interaction was positive
            is_friend_interaction (bool): Whether this is a friend interaction (reduced effect)
        """
        if self.is_high_integrity:
            return  # High integrity agents don't change integrity
        
        # Define base influence strength
        INFLUENCE_STRENGTH = 0.1
        
        # Reduce effect for friend interactions (1/16th of normal effect)
        if is_friend_interaction:
            INFLUENCE_STRENGTH *= 0.0625
        
        # Calculate change based on interaction type
        if same_opinion and is_positive:
            # Positive interaction with same opinion → increase integrity
            # Higher integrity agents are more influential
            change = INFLUENCE_STRENGTH * (1.0 - self.integrity)
            self.integrity = min(1.0, self.integrity + change)
        elif not same_opinion and is_positive:
            # Positive interaction with different opinion → decrease integrity
            change = INFLUENCE_STRENGTH * self.integrity
            self.integrity = max(0.0, self.integrity - change)
        elif same_opinion and not is_positive:
            # Negative interaction with same opinion → decrease integrity
            change = INFLUENCE_STRENGTH * self.integrity
            self.integrity = max(0.0, self.integrity - change)
        elif not same_opinion and not is_positive:
            # Negative interaction with different opinion → increase integrity
            change = INFLUENCE_STRENGTH * (1.0 - self.integrity)
            self.integrity = min(1.0, self.integrity + change)
    
    def _maybe_change_opinion(self, other_agent: 'Agent', is_friend_interaction: bool = False) -> None:
        """
        Maybe change opinion based on integrity level and other agent's influence.
        
        Args:
            other_agent (Agent): The agent that might influence this one
            is_friend_interaction (bool): Whether this is a friend interaction (reduced effect)
        """
        if self.is_high_integrity:
            return  # High integrity agents never change opinions
        
        # Calculate change probability based on:
        # 1. This agent's susceptibility (1.0 - integrity)
        # 2. Other agent's influence strength (integrity)
        susceptibility = 1.0 - self.integrity
        influence_strength = other_agent.integrity
        change_probability = susceptibility * influence_strength * 0.5  # Scale factor
        
        # Reduce effect for friend interactions (1/16th of normal effect)
        if is_friend_interaction:
            change_probability *= 0.0625
        
        if random.random() < change_probability:
            self.opinion = other_agent.opinion 

In [18]:
import numpy as np
from typing import List, Tuple
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from agent import Agent

class Environment:
    def __init__(self, width: int = 100, height: int = 100):
        """
        Initialize the grid environment.
        
        Args:
            width (int): Width of the grid
            height (int): Height of the grid
        """
        self.width = width
        self.height = height
        self.grid = np.zeros((height, width), dtype=int)  # Initialize with 0's to represent empty cell
        self.agents: List[Agent] = []
        
    def add_agent(self, agent: Agent) -> bool:
        """
        Add an agent to the environment at its current position.
        
        Args:
            agent (Agent): The agent to add
            
        Returns:
            bool: True if agent was successfully added, False otherwise
        """
        x, y = agent.position
        if 0 <= x < self.width and 0 <= y < self.height and self.grid[y, x] == 0:
            self.grid[y, x] = 1  # Mark cell as occupied
            self.agents.append(agent)
            return True
        return False
    
    def remove_agent(self, agent: Agent) -> None:
        """
        Remove an agent from the environment.
        
        Args:
            agent (Agent): The agent to remove
        """
        if agent in self.agents:
            x, y = agent.position
            self.grid[y, x] = 0  # Mark cell as empty
            self.agents.remove(agent)
    
    def validate_move(self, agent, new_position: Tuple[int, int]) -> bool:
        """
        Validate if a move is possible.
        
        Args:
            agent: The agent trying to move
            new_position (Tuple[int, int]): Proposed new position (x, y)
            
        Returns:
            bool: True if move is valid, False otherwise
        """
        x, y = new_position
        return (0 <= x < self.width and 
                0 <= y < self.height and 
                self.grid[y, x] == 0)
    
    def move_agent(self, agent: Agent, new_position: Tuple[int, int]) -> bool:
        """
        Move an agent to a new position if the move is valid.
        
        Args:
            agent (Agent): The agent to move
            new_position (Tuple[int, int]): New position (x, y)
            
        Returns:
            bool: True if move was successful, False otherwise
        """
        if self.validate_move(agent, new_position):
            old_x, old_y = agent.position
            x, y = new_position
            self.grid[old_y, old_x] = 0  # Clear old position
            self.grid[y, x] = 1  # Mark new position as occupied
            agent.position = new_position
            return True
        return False
    
    def get_empty_cells(self) -> List[Tuple[int, int]]:
        """
        Get a list of all empty cell positions.
        
        Returns:
            List[Tuple[int, int]]: List of (x, y) coordinates of empty cells
        """
        return [(x, y) for y in range(self.height) for x in range(self.width) 
                if self.grid[y, x] == 0]

    def step(self) -> None:
        """
        Run one step of the simulation where each agent:
        1. Moves to a random adjacent cell if possible
        2. Interacts with agents within their influence range (based on integrity)
        """
        # Shuffle agents to randomize order of movement and interaction
        random.shuffle(self.agents)
        
        # Move agents
        for agent in self.agents:
            proposed_move = agent.propose_move()
            self.move_agent(agent, proposed_move)
        
        # Shuffle again before interactions to ensure random order
        random.shuffle(self.agents)
        
        # Handle interactions with integrity-based range
        for agent in self.agents:
            x, y = agent.position
            
            # Calculate interaction range based on integrity (0-3 cells)
            max_range = min(3, int(agent.integrity * 3))
            
            # Check all cells within the agent's influence range
            for dx in range(-max_range, max_range + 1):
                for dy in range(-max_range, max_range + 1):
                    # Skip the agent's own cell
                    if dx == 0 and dy == 0:
                        continue
                    
                    nx, ny = x + dx, y + dy
                    
                    # Check if the cell is within grid bounds and occupied
                    if 0 <= nx < self.width and 0 <= ny < self.height and self.grid[ny, nx] == 1:
                        # Find the agent at this position (much faster with grid lookup)
                        for other_agent in self.agents:
                            if other_agent.position == (nx, ny):
                                agent.interact(other_agent)
                                break  # Found it, stop searching

    def get_opinion_distribution(self) -> dict:
        """
        Get the current distribution of opinions among agents.
        
        Returns:
            dict: Dictionary mapping opinions to counts
        """
        opinion_counts = {}
        for agent in self.agents:
            opinion_counts[agent.opinion] = opinion_counts.get(agent.opinion, 0) + 1
        return opinion_counts 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from IPython.display import HTML, display
from environment import Environment
from agent import Agent

def create_environment(width: int = 100, height: int = 100, num_agents: int = 1000, high_integrity_ratio: float = 0.1) -> Environment:
    """
    Create and initialize the environment with agents.
    
    Args:
        width (int): Width of the grid
        height (int): Height of the grid
        num_agents (int): Number of agents to create
        high_integrity_ratio (float): Fraction of agents that should be high integrity (0.0 to 1.0)
        
    Returns:
        Environment: The initialized environment
    """
    env = Environment(width, height)
    
    # Calculate number of high integrity agents
    num_high_integrity = int(num_agents * high_integrity_ratio)
    num_regular_agents = num_agents - num_high_integrity
    
    # Create agents with equal distribution of opinions
    opinions_per_agent = num_regular_agents // 3
    high_integrity_per_opinion = num_high_integrity // 3
    
    # Create regular agents
    for i in range(3):
        for _ in range(opinions_per_agent):
            # Get a random empty cell
            empty_cells = env.get_empty_cells()
            if empty_cells:
                position = empty_cells[np.random.randint(len(empty_cells))]
                agent = Agent(position, opinion=i, is_high_integrity=False)
                env.add_agent(agent)
    
    # Create high integrity agents
    for i in range(3):
        for _ in range(high_integrity_per_opinion):
            # Get a random empty cell
            empty_cells = env.get_empty_cells()
            if empty_cells:
                position = empty_cells[np.random.randint(len(empty_cells))]
                agent = Agent(position, opinion=i, is_high_integrity=True)
                env.add_agent(agent)
    
    return env

def visualize_environment(env: Environment, show: bool = True) -> None:
    """
    Visualize the current state of the environment using matplotlib.
    Empty cells are white, agents are colored by their opinion.
    
    Args:
        env (Environment): The environment to visualize
        show (bool): Whether to display the plot
    """
    # Create a grid where:
    # 0 = empty (white)
    # 1-3 = different opinions (different colors)
    vis_grid = np.zeros((env.height, env.width))
    
    # Map agent positions and opinions to the visualization grid
    for agent in env.agents:
        x, y = agent.position
        vis_grid[y, x] = agent.opinion + 1  # +1 because 0 is reserved for empty cells
    
    # Create custom colormap
    colors = ['white', 'red', 'blue', 'green']  # white for empty, then colors for opinions
    max_opinion = int(vis_grid.max())  # Convert to int for indexing
    cmap = ListedColormap(colors[:max_opinion + 1])
    
    # Create the plot
    plt.figure(figsize=(10, 10))
    plt.imshow(vis_grid, cmap=cmap, vmin=0, vmax=len(colors)-1)
    plt.grid(True, which='both', color='black', linewidth=0.5)
    plt.xticks(range(env.width))
    plt.yticks(range(env.height))
    
    # Add a legend
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Empty')]
    for i, color in enumerate(colors[1:], 1):
        legend_elements.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, 
                                           label=f'Opinion {i-1}'))
    plt.legend(handles=legend_elements, loc='upper right')
    
    plt.title('Environment State')
    if show:
        plt.show()
    plt.close()

def animate_simulation(env: Environment, steps: int, interval: int = 500) -> HTML:
    """
    Create an animation of the simulation.
    
    Args:
        env (Environment): The environment to simulate
        steps (int): Number of steps to simulate
        interval (int): Time between frames in milliseconds
        
    Returns:
        HTML: HTML representation of the animation for notebook display
    """
    # Create figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), 
                                  gridspec_kw={'width_ratios': [2, 1]})
    
    # Create custom colormap
    colors = ['white', 'red', 'blue', 'green']
    cmap = ListedColormap(colors)
    
    # Initialize the grid plot
    vis_grid = np.zeros((env.height, env.width))
    for agent in env.agents:
        x, y = agent.position
        vis_grid[y, x] = agent.opinion + 1
    
    img = ax1.imshow(vis_grid, cmap=cmap, vmin=0, vmax=len(colors)-1)
    ax1.grid(True, which='both', color='black', linewidth=0.5)
    # Remove tick marks and labels
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('Agent Positions')
    
    # Add legend to grid plot
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Empty')]
    for i, color in enumerate(colors[1:], 1):
        legend_elements.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, 
                                           label=f'Opinion {i-1}'))
    ax1.legend(handles=legend_elements, loc='upper right')
    
    # Initialize the time series plot
    opinion_dist = env.get_opinion_distribution()
    time_data = [0]  # Time points
    opinion_data = {0: [opinion_dist.get(0, 0)],  # Opinion 0 counts
                   1: [opinion_dist.get(1, 0)],  # Opinion 1 counts
                   2: [opinion_dist.get(2, 0)]}  # Opinion 2 counts
    
    # Create lines for each opinion
    lines = []
    for i in range(3):
        line, = ax2.plot(time_data, opinion_data[i], color=colors[i+1], 
                       label=f'Opinion {i}', linewidth=2)
        lines.append(line)
    
    ax2.set_xlabel('Step')
    ax2.set_ylabel('Number of Agents')
    ax2.set_title('Opinion Distribution Over Time')
    ax2.set_ylim(0, len(env.agents))
    ax2.set_xlim(0, steps)
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.legend()
    
    # Add opinion distribution text
    opinion_text = ax1.text(0.02, 0.98, '', transform=ax1.transAxes, 
                          verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Add step count text
    step_text = ax1.text(0.5, 0.02, '', transform=ax1.transAxes, 
                        horizontalalignment='center', fontsize=12, fontweight='bold',
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    def update(frame):
        env.step()
        # Update grid
        vis_grid = np.zeros((env.height, env.width))
        for agent in env.agents:
            x, y = agent.position
            vis_grid[y, x] = agent.opinion + 1
        img.set_array(vis_grid)
        
        # Update opinion distribution
        opinion_dist = env.get_opinion_distribution()
        time_data.append(frame + 1)
        for i in range(3):
            opinion_data[i].append(opinion_dist.get(i, 0))
            lines[i].set_data(time_data, opinion_data[i])
        
        # Dynamically rescale y-axis based on current data
        all_values = []
        for i in range(3):
            all_values.extend(opinion_data[i])
        
        if all_values:  # Only rescale if we have data
            min_val = min(all_values)
            max_val = max(all_values)
            margin = (max_val - min_val) * 0.1  # 10% margin
            if margin == 0:  # If all values are the same
                margin = 1
            ax2.set_ylim(max(0, min_val - margin), max_val + margin)
        
        # Update opinion text
        opinion_text.set_text('\n'.join([f'Opinion {k}: {v} agents' 
                                       for k, v in sorted(opinion_dist.items())]))
        
        # Update step count text
        step_text.set_text(f'Step {frame + 1}/{steps}')
        
        # Stop animation if we've reached the last frame
        if frame >= steps - 1:
            anim.event_source.stop()
        
        return [img, opinion_text, step_text] + lines
    
    anim = animation.FuncAnimation(fig, update, frames=steps, interval=interval, blit=True)
    plt.close(fig)  # Close the figure to prevent display issues
    return HTML(anim.to_jshtml())  # Return HTML for notebook display

def main():
    # Create and initialize the environment
    env = create_environment(width=50, height=50, num_agents=1000, high_integrity_ratio=0.1)
    
    # Run the simulation with animation
    anim_html = animate_simulation(env, steps=400, interval=50)
    display(anim_html)  # Display the animation in the notebook

if __name__ == "__main__":
    main() 

Final iteration

In [47]:
from typing import Tuple, List
import random

class Agent:
    def __init__(self, position: Tuple[int, int], opinion: int = 0, is_high_integrity: bool = False):
        """
        Initialize an agent with a position, opinion, and integrity.
        
        Args:
            position (Tuple[int, int]): Initial position (x, y)
            opinion (int): Initial opinion (0, 1, or 2)
            is_high_integrity (bool): Whether this agent has fixed high integrity
        """
        self.position = position
        self.opinion = opinion
        self.is_high_integrity = is_high_integrity
        self.friends: List['Agent'] = []  # List of friend agents
        
        # Schedule-related attributes
        self.is_student = random.choice([True, False])  # Half are students, half are workers
        self.current_location = 'home'  # Current structure type
        self.target_location = 'home'   # Where they're trying to go
        self.schedule_step = 0          # Current step in their daily schedule
        
        # Assign specific structures (will be set by environment)
        self.assigned_home = None
        self.assigned_work_or_school = None
        self.assigned_leisure = None
        
        # Set integrity based on agent type
        if is_high_integrity:
            self.integrity = 1.0  # 100% integrity, never changes
        else:
            self.integrity = random.uniform(0.3, 0.7)  # Random initial integrity
    
    def add_friend(self, friend: 'Agent') -> None:
        """
        Add a friend to this agent's social circle.
        
        Args:
            friend (Agent): The agent to add as a friend
        """
        if friend not in self.friends and friend != self:
            self.friends.append(friend)
            # Make it bidirectional
            if self not in friend.friends:
                friend.friends.append(self)
    
    def remove_friend(self, friend: 'Agent') -> None:
        """
        Remove a friend from this agent's social circle.
        
        Args:
            friend (Agent): The agent to remove as a friend
        """
        if friend in self.friends:
            self.friends.remove(friend)
            # Make it bidirectional
            if self in friend.friends:
                friend.friends.remove(self)
    
    def add_friends(self, all_agents: List['Agent'], num_friends: int = 8) -> None:
        """
        Add friends to this agent with the same opinion.
        
        Args:
            all_agents (List[Agent]): All agents in the simulation
            num_friends (int): Maximum number of friends to add
        """
        # Filter out self and existing friends, and only include agents with same opinion
        available_agents = [agent for agent in all_agents 
                          if agent != self and agent not in self.friends and agent.opinion == self.opinion]
        
        # Add friends (all will have same opinion)
        friends_added = 0
        while friends_added < num_friends and available_agents:
            # Pick a random agent with same opinion
            friend = random.choice(available_agents)
            available_agents.remove(friend)
            
            # Add friend (bidirectional)
            self.add_friend(friend)
            friends_added += 1

    def propose_move(self, environment) -> Tuple[int, int]:
        """
        Propose a move toward the target location based on schedule.
        
        Args:
            environment: The environment containing structure bounds
            
        Returns:
            Tuple[int, int]: Proposed new position (x, y)
        """
        # Check if we're already in the target structure
        if self._is_in_target_structure(environment):
            return self._random_move_within_structure(environment)
        
        # Move toward target structure
        return self._move_toward_structure(environment)
    
    def _is_in_target_structure(self, environment) -> bool:
        """
        Check if the agent is currently in their target structure.
        
        Args:
            environment: The environment containing structure bounds
            
        Returns:
            bool: True if agent is in target structure
        """
        if self.target_location == 'home' and self.assigned_home:
            return self.position in self.assigned_home
        elif self.target_location in ['work', 'school'] and self.assigned_work_or_school:
            return self.position in self.assigned_work_or_school
        elif self.target_location == 'leisure' and self.assigned_leisure:
            return self.position in self.assigned_leisure
        else:
            # Fallback to environment bounds if no assignment
            if self.target_location == 'home':
                return self.position in environment.home_bounds
            elif self.target_location == 'work':
                return self.position in environment.work_bounds
            elif self.target_location == 'school':
                return self.position in environment.school_bounds
            elif self.target_location == 'leisure':
                return self.position in environment.leisure_bounds
            return True  # Default to staying put
    
    def _move_toward_structure(self, environment) -> Tuple[int, int]:
        """
        Move toward the target structure area.
        
        Args:
            environment: The environment containing structure bounds
            
        Returns:
            Tuple[int, int]: Proposed new position
        """
        # Get the bounds of the target structure (use assigned structure if available)
        bounds = None
        if self.target_location == 'home' and self.assigned_home:
            bounds = self.assigned_home
        elif self.target_location in ['work', 'school'] and self.assigned_work_or_school:
            bounds = self.assigned_work_or_school
        elif self.target_location == 'leisure' and self.assigned_leisure:
            bounds = self.assigned_leisure
        else:
            # Fallback to environment bounds
            if self.target_location == 'home':
                bounds = environment.home_bounds
            elif self.target_location == 'work':
                bounds = environment.work_bounds
            elif self.target_location == 'school':
                bounds = environment.school_bounds
            elif self.target_location == 'leisure':
                bounds = environment.leisure_bounds
        
        if not bounds:
            return self.position
        
        # Find the center of the target structure
        center_x = sum(x for x, y in bounds) // len(bounds)
        center_y = sum(y for x, y in bounds) // len(bounds)
        
        # Move toward the center of the structure
        return self._move_toward_target((center_x, center_y))
    
    def _move_toward_target(self, target_pos: Tuple[int, int]) -> Tuple[int, int]:
        """
        Move toward a target position.
        
        Args:
            target_pos (Tuple[int, int]): Target position to move toward
            
        Returns:
            Tuple[int, int]: Proposed new position
        """
        x, y = self.position
        tx, ty = target_pos
        
        # Calculate direction toward target
        dx = 0
        dy = 0
        
        if x < tx:
            dx = 1
        elif x > tx:
            dx = -1
            
        if y < ty:
            dy = 1
        elif y > ty:
            dy = -1
        
        # Propose move in the direction of the target
        new_x = x + dx
        new_y = y + dy
        
        return (new_x, new_y)
    
    def _random_move_within_structure(self, environment) -> Tuple[int, int]:
        """
        Move randomly within the current structure.
        
        Args:
            environment: The environment containing structure bounds
            
        Returns:
            Tuple[int, int]: Proposed new position
        """
        # Get bounds for current target location (use assigned structure if available)
        bounds = None
        if self.target_location == 'home' and self.assigned_home:
            bounds = self.assigned_home
        elif self.target_location in ['work', 'school'] and self.assigned_work_or_school:
            bounds = self.assigned_work_or_school
        elif self.target_location == 'leisure' and self.assigned_leisure:
            bounds = self.assigned_leisure
        else:
            # Fallback to environment bounds
            if self.target_location == 'home':
                bounds = environment.home_bounds
            elif self.target_location == 'work':
                bounds = environment.work_bounds
            elif self.target_location == 'school':
                bounds = environment.school_bounds
            elif self.target_location == 'leisure':
                bounds = environment.leisure_bounds
        
        if bounds:
            # Filter out the current position to avoid staying in the same place
            available_positions = [pos for pos in bounds if pos != self.position]
            if available_positions:
                return random.choice(available_positions)
            else:
                # If no other positions available, stay put
                return self.position
        else:
            # Fallback to random adjacent move
            x, y = self.position
            possible_moves = []
            for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
                new_x, new_y = x + dx, y + dy
                # Only add valid moves (within bounds and not current position)
                if (0 <= new_x < environment.width and 
                    0 <= new_y < environment.height and 
                    (new_x, new_y) != self.position):
                    possible_moves.append((new_x, new_y))
            
            if possible_moves:
                return random.choice(possible_moves)
            else:
                return self.position
    
    def interact(self, other_agent: 'Agent', is_friend_interaction: bool = False) -> None:
        """
        Interact with another agent, potentially changing opinions and integrity.
        
        Args:
            other_agent (Agent): The agent to interact with
            is_friend_interaction (bool): Whether this is a friend interaction (reduced effect)
        """
        # Determine if this is a positive or negative interaction (70% positive, 30% negative)
        is_positive = random.random() < 0.7
        
        # Determine if agents have the same opinion
        same_opinion = (self.opinion == other_agent.opinion)
        
        # Update integrity based on interaction type
        # High integrity agents can influence others but don't change their own integrity
        if not self.is_high_integrity:
            self._update_integrity(same_opinion, is_positive, is_friend_interaction)
        
        if not other_agent.is_high_integrity:
            other_agent._update_integrity(same_opinion, is_positive, is_friend_interaction)
        
        # Check if opinions should change based on integrity
        # High integrity agents can influence others but don't change their own opinions
        if not self.is_high_integrity:
            self._maybe_change_opinion(other_agent, is_friend_interaction)
        
        if not other_agent.is_high_integrity:
            other_agent._maybe_change_opinion(self, is_friend_interaction)
    
    def _update_integrity(self, same_opinion: bool, is_positive: bool, is_friend_interaction: bool = False) -> None:
        """
        Update agent's integrity based on interaction type.
        
        Args:
            same_opinion (bool): Whether the interaction was with same opinion
            is_positive (bool): Whether the interaction was positive
            is_friend_interaction (bool): Whether this is a friend interaction (reduced effect)
        """
        if self.is_high_integrity:
            return  # High integrity agents don't change integrity
        
        # Define base influence strength
        INFLUENCE_STRENGTH = 0.1
        
        # Reduce effect for friend interactions (1/16th of normal effect)
        if is_friend_interaction:
            INFLUENCE_STRENGTH *= 0.0625
        
        # Calculate change based on interaction type
        if same_opinion and is_positive:
            # Positive interaction with same opinion → increase integrity
            # Higher integrity agents are more influential
            change = INFLUENCE_STRENGTH * (1.0 - self.integrity)
            self.integrity = min(1.0, self.integrity + change)
        elif not same_opinion and is_positive:
            # Positive interaction with different opinion → decrease integrity
            change = INFLUENCE_STRENGTH * self.integrity
            self.integrity = max(0.0, self.integrity - change)
        elif same_opinion and not is_positive:
            # Negative interaction with same opinion → decrease integrity
            change = INFLUENCE_STRENGTH * self.integrity
            self.integrity = max(0.0, self.integrity - change)
        elif not same_opinion and not is_positive:
            # Negative interaction with different opinion → increase integrity
            change = INFLUENCE_STRENGTH * (1.0 - self.integrity)
            self.integrity = min(1.0, self.integrity + change)
    
    def _maybe_change_opinion(self, other_agent: 'Agent', is_friend_interaction: bool = False) -> None:
        """
        Maybe change opinion based on integrity level and other agent's influence.
        
        Args:
            other_agent (Agent): The agent that might influence this one
            is_friend_interaction (bool): Whether this is a friend interaction (reduced effect)
        """
        if self.is_high_integrity:
            return  # High integrity agents never change opinions
        
        # Calculate change probability based on:
        # 1. This agent's susceptibility (1.0 - integrity)
        # 2. Other agent's influence strength (integrity)
        susceptibility = 1.0 - self.integrity
        influence_strength = other_agent.integrity
        change_probability = susceptibility * influence_strength * 0.5  # Scale factor
        
        # Reduce effect for friend interactions (1/16th of normal effect)
        if is_friend_interaction:
            change_probability *= 0.0625
        
        if random.random() < change_probability:
            self.opinion = other_agent.opinion 

    def update_schedule(self, step: int) -> None:
        """
        Update the agent's schedule based on the current simulation step.
        
        Args:
            step (int): Current simulation step
        """
        # Daily schedule: 120 steps per day (longer day cycle for more realistic movement)
        day_step = step % 120
        
        if self.is_student:
            # Student schedule (longer periods)
            if 10 <= day_step < 20:  # Morning: go to school
                self.target_location = 'school'
            elif 20 <= day_step < 80:  # School hours: stay at school (60 steps)
                self.target_location = 'school'
            elif 80 <= day_step < 100:  # Afternoon: go to leisure (20 steps)
                self.target_location = 'leisure'
            elif 100 <= day_step < 110:  # Evening: go home
                self.target_location = 'home'
            else:  # Night: stay home
                self.target_location = 'home'
        else:
            # Worker schedule (longer periods)
            if 10 <= day_step < 20:  # Morning: go to work
                self.target_location = 'work'
            elif 20 <= day_step < 80:  # Work hours: stay at work (60 steps)
                self.target_location = 'work'
            elif 80 <= day_step < 100:  # Afternoon: go to leisure (20 steps)
                self.target_location = 'leisure'
            elif 100 <= day_step < 110:  # Evening: go home
                self.target_location = 'home'
            else:  # Night: stay home
                self.target_location = 'home'

    def assign_structures(self, home_bounds, work_school_bounds, leisure_bounds):
        """
        Assign specific structure areas to this agent.
        
        Args:
            home_bounds: List of positions for assigned home structure
            work_school_bounds: List of positions for assigned work/school structure
            leisure_bounds: List of positions for assigned leisure structure
        """
        self.assigned_home = home_bounds
        self.assigned_work_or_school = work_school_bounds
        self.assigned_leisure = leisure_bounds 

In [48]:
import numpy as np
from typing import List, Tuple
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from agent import Agent

class Environment:
    def __init__(self, width: int = 100, height: int = 100):
        """
        Initialize the grid environment.
        
        Args:
            width (int): Width of the grid
            height (int): Height of the grid
        """
        self.width = width
        self.height = height
        self.grid = np.zeros((height, width), dtype=int)  # Initialize with 0's to represent empty cell
        self.agents: List[Agent] = []
        self.step_count = 0  # Track simulation steps
        
        # Define structure bounds
        self.home_bounds = []
        self.work_bounds = []
        self.school_bounds = []
        self.leisure_bounds = []
        
        # Set up structure bounds
        self._setup_structure_bounds()
        
    def _setup_structure_bounds(self):
        """Set up the bounds for different structure types distributed across the grid."""
        # Clear existing bounds
        self.home_bounds = []
        self.work_bounds = []
        self.school_bounds = []
        self.leisure_bounds = []
        
        # Define structure size (larger structures for better visibility)
        structure_size = 12
        spacing = 4  # Space between structures
        
        # HOME STRUCTURES - Top half of the grid
        home_positions = [
            (10, 10), (35, 8), (60, 12), (85, 10),      # Top row
            (15, 30), (45, 28), (75, 32), (90, 35),     # Second row
            (5, 50), (30, 48), (55, 52), (80, 50)       # Third row
        ]
        
        for center_x, center_y in home_positions:
            if center_x + structure_size//2 < self.width and center_y + structure_size//2 < self.height:
                start_x = max(0, center_x - structure_size//2)
                start_y = max(0, center_y - structure_size//2)
                
                for x in range(start_x, min(start_x + structure_size, self.width)):
                    for y in range(start_y, min(start_y + structure_size, self.height)):
                        self.home_bounds.append((x, y))
        
        # WORK STRUCTURES - Bottom left quarter
        work_positions = [
            (15, 70), (40, 75), (20, 90),               # Concentrated in bottom left
            (45, 85), (10, 85), (35, 65)
        ]
        
        for center_x, center_y in work_positions:
            if center_x + structure_size//2 < self.width and center_y + structure_size//2 < self.height:
                start_x = max(0, center_x - structure_size//2)
                start_y = max(0, center_y - structure_size//2)
                
                for x in range(start_x, min(start_x + structure_size, self.width)):
                    for y in range(start_y, min(start_y + structure_size, self.height)):
                        self.work_bounds.append((x, y))
        
        # SCHOOL STRUCTURES - Bottom right quarter  
        school_positions = [
            (65, 70), (85, 75), (70, 90),               # Concentrated in bottom right
            (90, 85), (55, 85), (80, 65)
        ]
        
        for center_x, center_y in school_positions:
            if center_x + structure_size//2 < self.width and center_y + structure_size//2 < self.height:
                start_x = max(0, center_x - structure_size//2)
                start_y = max(0, center_y - structure_size//2)
                
                for x in range(start_x, min(start_x + structure_size, self.width)):
                    for y in range(start_y, min(start_y + structure_size, self.height)):
                        self.school_bounds.append((x, y))
        
        # LEISURE STRUCTURES - Scattered throughout but avoiding residential areas
        leisure_size = 8  # Smaller leisure structures
        leisure_positions = [
            (25, 65),   # Between work and home areas
            (75, 45),   # Right side, between home and school
            (50, 75),   # Center bottom
            (20, 40),   # Left side
            (85, 40),   # Right side
            (50, 20),   # Center top
            (10, 65),   # Far left
            (90, 65),   # Far right
        ]
        
        for center_x, center_y in leisure_positions:
            if center_x + leisure_size//2 < self.width and center_y + leisure_size//2 < self.height:
                start_x = max(0, center_x - leisure_size//2)
                start_y = max(0, center_y - leisure_size//2)
                
                for x in range(start_x, min(start_x + leisure_size, self.width)):
                    for y in range(start_y, min(start_y + leisure_size, self.height)):
                        self.leisure_bounds.append((x, y))
    
    def add_agent(self, agent: Agent) -> bool:
        """
        Add an agent to the environment at its current position.
        
        Args:
            agent (Agent): The agent to add
            
        Returns:
            bool: True if agent was successfully added, False otherwise
        """
        x, y = agent.position
        if 0 <= x < self.width and 0 <= y < self.height and self.grid[y, x] == 0:
            self.grid[y, x] = 1  # Mark cell as occupied
            self.agents.append(agent)
            return True
        return False
    
    def remove_agent(self, agent: Agent) -> None:
        """
        Remove an agent from the environment.
        
        Args:
            agent (Agent): The agent to remove
        """
        if agent in self.agents:
            x, y = agent.position
            self.grid[y, x] = 0  # Mark cell as empty
            self.agents.remove(agent)
    
    def validate_move(self, agent, new_position: Tuple[int, int]) -> bool:
        """
        Validate if a move is possible.
        
        Args:
            agent: The agent trying to move
            new_position (Tuple[int, int]): Proposed new position (x, y)
            
        Returns:
            bool: True if move is valid, False otherwise
        """
        x, y = new_position
        return (0 <= x < self.width and 
                0 <= y < self.height and 
                self.grid[y, x] == 0)
    
    def move_agent(self, agent: Agent, new_position: Tuple[int, int]) -> bool:
        """
        Move an agent to a new position if the move is valid.
        
        Args:
            agent (Agent): The agent to move
            new_position (Tuple[int, int]): New position (x, y)
            
        Returns:
            bool: True if move was successful, False otherwise
        """
        if self.validate_move(agent, new_position):
            old_x, old_y = agent.position
            x, y = new_position
            self.grid[old_y, old_x] = 0  # Clear old position
            self.grid[y, x] = 1  # Mark new position as occupied
            agent.position = new_position
            return True
        return False
    
    def get_empty_cells(self) -> List[Tuple[int, int]]:
        """
        Get a list of all empty cell positions.
        
        Returns:
            List[Tuple[int, int]]: List of (x, y) coordinates of empty cells
        """
        return [(x, y) for y in range(self.height) for x in range(self.width) 
                if self.grid[y, x] == 0]

    def step(self) -> None:
        """
        Run one step of the simulation where each agent:
        1. Updates their schedule based on current step
        2. Moves toward their target location based on schedule
        3. Interacts with a subset of their friends (reduced effect)
        4. Interacts with agents within their influence range (based on integrity)
        """
        # Shuffle agents to randomize order of movement and interaction
        random.shuffle(self.agents)
        
        # Update schedules and move agents
        for agent in self.agents:
            agent.update_schedule(self.step_count)
            proposed_move = agent.propose_move(self)
            self.move_agent(agent, proposed_move)
        
        # Shuffle again before interactions to ensure random order
        random.shuffle(self.agents)
        
        # Handle friend interactions (each agent interacts with 1-3 random friends)
        for agent in self.agents:
            if agent.friends:
                # Pick 1-3 random friends to interact with this step
                num_friend_interactions = random.randint(1, min(3, len(agent.friends)))
                friends_to_interact_with = random.sample(agent.friends, num_friend_interactions)
                for friend in friends_to_interact_with:
                    agent.interact(friend, is_friend_interaction=True)
        
        # Shuffle again before spatial interactions
        random.shuffle(self.agents)
        
        # Handle spatial interactions with integrity-based range
        for agent in self.agents:
            x, y = agent.position
            
            # Calculate interaction range based on integrity (0-3 cells)
            max_range = min(3, int(agent.integrity * 3))
            
            # Check all cells within the agent's influence range
            for dx in range(-max_range, max_range + 1):
                for dy in range(-max_range, max_range + 1):
                    # Skip the agent's own cell
                    if dx == 0 and dy == 0:
                        continue
                    
                    nx, ny = x + dx, y + dy
                    
                    # Check if the cell is within grid bounds and occupied
                    if 0 <= nx < self.width and 0 <= ny < self.height and self.grid[ny, nx] == 1:
                        # Find the agent at this position (much faster with grid lookup)
                        for other_agent in self.agents:
                            if other_agent.position == (nx, ny):
                                agent.interact(other_agent, is_friend_interaction=False)
                                break  # Found it, stop searching
        
        # Increment step counter
        self.step_count += 1

    def get_opinion_distribution(self) -> dict:
        """
        Get the current distribution of opinions among agents.
        
        Returns:
            dict: Dictionary mapping opinions to counts
        """
        opinion_counts = {}
        for agent in self.agents:
            opinion_counts[agent.opinion] = opinion_counts.get(agent.opinion, 0) + 1
        return opinion_counts 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from environment import Environment
from agent import Agent

def create_environment(width: int = 100, height: int = 100, num_agents: int = 1000, high_integrity_ratio: float = 0.1) -> Environment:
    """
    Create and initialize the environment with agents.
    
    Args:
        width (int): Width of the grid
        height (int): Height of the grid
        num_agents (int): Number of agents to create
        high_integrity_ratio (float): Fraction of agents that should be high integrity (0.0 to 1.0)
        
    Returns:
        Environment: The initialized environment
    """
    env = Environment(width, height)
    
    # Calculate number of high integrity agents
    num_high_integrity = int(num_agents * high_integrity_ratio)
    num_regular_agents = num_agents - num_high_integrity
    
    # Create agents with equal distribution of opinions
    opinions_per_agent = num_regular_agents // 3
    high_integrity_per_opinion = num_high_integrity // 3
    
    # Create structure groups for assignment
    def divide_structure_into_groups(bounds, num_groups):
        """Divide structure bounds into groups for assignment."""
        if not bounds or num_groups <= 0:
            return []
        
        # Sort bounds to group nearby positions
        sorted_bounds = sorted(bounds)
        group_size = len(sorted_bounds) // num_groups
        groups = []
        
        for i in range(num_groups):
            start_idx = i * group_size
            if i == num_groups - 1:  # Last group gets remaining positions
                end_idx = len(sorted_bounds)
            else:
                end_idx = (i + 1) * group_size
            groups.append(sorted_bounds[start_idx:end_idx])
        
        return groups
    
    # Divide structures into groups
    home_groups = divide_structure_into_groups(env.home_bounds, 12)  # 12 home areas
    work_groups = divide_structure_into_groups(env.work_bounds, 6)   # 6 work areas
    school_groups = divide_structure_into_groups(env.school_bounds, 6) # 6 school areas
    leisure_groups = divide_structure_into_groups(env.leisure_bounds, 8) # 8 leisure areas
    
    all_agents = []
    
    # Create regular agents
    for i in range(3):
        for _ in range(opinions_per_agent):
            # Get a random empty cell
            empty_cells = env.get_empty_cells()
            if empty_cells:
                position = empty_cells[np.random.randint(len(empty_cells))]
                agent = Agent(position, opinion=i, is_high_integrity=False)
                all_agents.append(agent)
    
    # Create high integrity agents
    for i in range(3):
        for _ in range(high_integrity_per_opinion):
            # Get a random empty cell
            empty_cells = env.get_empty_cells()
            if empty_cells:
                position = empty_cells[np.random.randint(len(empty_cells))]
                agent = Agent(position, opinion=i, is_high_integrity=True)
                all_agents.append(agent)
    
    # Assign structures to agents
    for idx, agent in enumerate(all_agents):
        # Assign home (cycle through available home groups)
        if home_groups:
            home_group = home_groups[idx % len(home_groups)]
            agent.assigned_home = home_group
        
        # Assign work or school based on agent type
        if agent.is_student and school_groups:
            school_group = school_groups[idx % len(school_groups)]
            agent.assigned_work_or_school = school_group
        elif work_groups:
            work_group = work_groups[idx % len(work_groups)]
            agent.assigned_work_or_school = work_group
        
        # Assign leisure (cycle through available leisure groups)
        if leisure_groups:
            leisure_group = leisure_groups[idx % len(leisure_groups)]
            agent.assigned_leisure = leisure_group
        
        # Add agent to environment
        env.add_agent(agent)
    
    # Add friends to all agents after they are created
    for agent in env.agents:
        agent.add_friends(env.agents, num_friends=8)
    
    return env

def visualize_environment(env: Environment, show: bool = True) -> None:
    """
    Visualize the current state of the environment using matplotlib.
    Empty cells are white, agents are colored by their opinion.
    
    Args:
        env (Environment): The environment to visualize
        show (bool): Whether to display the plot
    """
    # Create a grid where:
    # 0 = empty (white)
    # 1-3 = different opinions (different colors)
    vis_grid = np.zeros((env.height, env.width))
    
    # Map agent positions and opinions to the visualization grid
    for agent in env.agents:
        x, y = agent.position
        vis_grid[y, x] = agent.opinion + 1  # +1 because 0 is reserved for empty cells
    
    # Create custom colormap
    colors = ['white', 'red', 'blue', 'green']  # white for empty, then colors for opinions
    max_opinion = int(vis_grid.max())  # Convert to int for indexing
    cmap = ListedColormap(colors[:max_opinion + 1])
    
    # Create the plot
    plt.figure(figsize=(10, 10))
    plt.imshow(vis_grid, cmap=cmap, vmin=0, vmax=len(colors)-1)
    plt.grid(True, which='both', color='black', linewidth=0.5)
    plt.xticks(range(env.width))
    plt.yticks(range(env.height))
    
    # Add a legend
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Empty')]
    for i, color in enumerate(colors[1:], 1):
        legend_elements.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, 
                                           label=f'Opinion {i-1}'))
    plt.legend(handles=legend_elements, loc='upper right')
    
    plt.title('Environment State')
    if show:
        plt.show()
    plt.close()

def animate_simulation(env: Environment, steps: int, interval: int = 500) -> None:
    """
    Create an animation of the simulation.
    
    Args:
        env (Environment): The environment to simulate
        steps (int): Number of steps to simulate
        interval (int): Time between frames in milliseconds
    """
    # Create figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), 
                                  gridspec_kw={'width_ratios': [2, 1]})
    
    # Create custom colormap
    colors = ['white', 'red', 'blue', 'green']
    cmap = ListedColormap(colors)
    
    # Initialize the grid plot
    vis_grid = np.zeros((env.height, env.width))
    for agent in env.agents:
        x, y = agent.position
        vis_grid[y, x] = agent.opinion + 1
    
    img = ax1.imshow(vis_grid, cmap=cmap, vmin=0, vmax=len(colors)-1)
    ax1.grid(True, which='both', color='black', linewidth=0.5)
    # Remove tick marks and labels
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('Agent Positions')
    
    # Add legend to grid plot
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Empty')]
    for i, color in enumerate(colors[1:], 1):
        legend_elements.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, 
                                           label=f'Opinion {i-1}'))
    ax1.legend(handles=legend_elements, loc='upper right')
    
    # Initialize the time series plot
    opinion_dist = env.get_opinion_distribution()
    time_data = [0]  # Time points
    opinion_data = {0: [opinion_dist.get(0, 0)],  # Opinion 0 counts
                   1: [opinion_dist.get(1, 0)],  # Opinion 1 counts
                   2: [opinion_dist.get(2, 0)]}  # Opinion 2 counts
    
    # Create lines for each opinion
    lines = []
    for i in range(3):
        line, = ax2.plot(time_data, opinion_data[i], color=colors[i+1], 
                       label=f'Opinion {i}', linewidth=2)
        lines.append(line)
    
    ax2.set_xlabel('Step')
    ax2.set_ylabel('Number of Agents')
    ax2.set_title('Opinion Distribution Over Time')
    ax2.set_ylim(0, len(env.agents))
    ax2.set_xlim(0, steps)
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.legend()
    
    # Add opinion distribution text
    opinion_text = ax1.text(0.02, 0.98, '', transform=ax1.transAxes, 
                          verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Add step count text
    step_text = ax1.text(0.5, 0.02, '', transform=ax1.transAxes, 
                        horizontalalignment='center', fontsize=12, fontweight='bold',
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Add schedule information text
    schedule_text = ax1.text(0.02, 0.02, '', transform=ax1.transAxes, 
                           verticalalignment='bottom', fontsize=10,
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    def update(frame):
        env.step()
        # Update grid
        vis_grid = np.zeros((env.height, env.width))
        for agent in env.agents:
            x, y = agent.position
            vis_grid[y, x] = agent.opinion + 1
        img.set_array(vis_grid)
        
        # Update opinion distribution
        opinion_dist = env.get_opinion_distribution()
        time_data.append(frame + 1)
        for i in range(3):
            opinion_data[i].append(opinion_dist.get(i, 0))
            lines[i].set_data(time_data, opinion_data[i])
        
        # Dynamically rescale y-axis based on current data
        all_values = []
        for i in range(3):
            all_values.extend(opinion_data[i])
        
        if all_values:  # Only rescale if we have data
            min_val = min(all_values)
            max_val = max(all_values)
            margin = (max_val - min_val) * 0.1  # 10% margin
            if margin == 0:  # If all values are the same
                margin = 1
            ax2.set_ylim(max(0, min_val - margin), max_val + margin)
        
        # Update opinion text
        opinion_text.set_text('\n'.join([f'Opinion {k}: {v} agents' 
                                       for k, v in sorted(opinion_dist.items())]))
        
        # Update step count text
        step_text.set_text(f'Step {frame + 1}/{steps}')
        
        # Update schedule information
        day_step = frame % 120  # Updated to match new day cycle
        day_number = frame // 120 + 1  # Which day we're on
        # Convert to 24-hour format (120 steps = 24 hours, so 1 step = 0.2 hours = 12 minutes)
        hour = int((day_step / 120) * 24)
        minute = int(((day_step / 120) * 24 - hour) * 60)
        time_str = f"Day {day_number}, {hour:02d}:{minute:02d}"
        
        # Count agents in each location
        location_counts = {'home': 0, 'work': 0, 'school': 0, 'leisure': 0}
        for agent in env.agents:
            location_counts[agent.target_location] += 1
        
        schedule_info = f"Time: {time_str}\n"
        schedule_info += f"Step: {day_step}/120\n"
        schedule_info += f"Home: {location_counts['home']}\n"
        schedule_info += f"Work: {location_counts['work']}\n"
        schedule_info += f"School: {location_counts['school']}\n"
        schedule_info += f"Leisure: {location_counts['leisure']}"
        schedule_text.set_text(schedule_info)
        
        # Stop animation if we've reached the last frame
        if frame >= steps - 1:
            anim.event_source.stop()
        
        return [img, opinion_text, step_text, schedule_text] + lines
    
    anim = animation.FuncAnimation(fig, update, frames=steps, interval=interval, blit=True)
    
    plt.show()

def main():
    # Create and initialize the environment
    env = create_environment(width=100, height=100, num_agents=1200, high_integrity_ratio=0.1)
    
    # Run the simulation with animation (500 steps = ~4 full days)
    animate_simulation(env, steps=500, interval=50)

if __name__ == "__main__":
    main() 

AttributeError: 'Environment' object has no attribute 'home_bounds'